In [1]:
import pandas as pd

# Source: Addressing Markeiting Bias in Product Recommendations - Mengting Wan, Jianmo Ni, Rishabh Misra, Julian McAuley - WSDM, 2020
# https://github.com/MengtingWan/marketBias
# https://github.com/Kal-Lemma/Clothes-Recommendation-System/blob/master/modcloth_final_data.json
# https://www.kaggle.com/code/agrawaladitya/step-by-step-data-preprocessing-eda/data
df = pd.read_json("../../data/recommendations/modcloth_final_data.json", lines=True)

In [2]:
df.head()

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,user_name,length,fit,user_id,shoe size,shoe width,review_summary,review_text
0,123373,29.0,7,5.0,d,38.0,34.0,new,36,5ft 6in,Emily,just right,small,991571,NaN,NaN,NaN,NaN
1,123373,31.0,13,3.0,b,30.0,36.0,new,NaN,5ft 2in,sydneybraden2001,just right,small,587883,NaN,NaN,NaN,NaN
2,123373,30.0,7,2.0,b,NaN,32.0,new,NaN,5ft 7in,Ugggh,slightly long,small,395665,9.0,NaN,NaN,NaN
3,123373,NaN,21,5.0,dd/e,NaN,NaN,new,NaN,NaN,alexmeyer626,just right,fit,875643,NaN,NaN,NaN,NaN
4,123373,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,dberrones1,slightly long,small,944840,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 82790 entries, 0 to 82789
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   item_id         82790 non-null  int64  
 1   waist           2882 non-null   float64
 2   size            82790 non-null  int64  
 3   quality         82722 non-null  float64
 4   cup size        76535 non-null  str    
 5   hips            56064 non-null  float64
 6   bra size        76772 non-null  float64
 7   category        82790 non-null  str    
 8   bust            11854 non-null  str    
 9   height          81683 non-null  str    
 10  user_name       82790 non-null  str    
 11  length          82755 non-null  str    
 12  fit             82790 non-null  str    
 13  user_id         82790 non-null  int64  
 14  shoe size       27915 non-null  float64
 15  shoe width      18607 non-null  str    
 16  review_summary  76065 non-null  str    
 17  review_text     76065 non-null  str    
dt

In [4]:
df = df[~df["user_id"].isna()]
df = df[~df["item_id"].isna()]
df = df[~df["quality"].isna()]
df = df[~df["review_text"].isna()]
df = df.reset_index()

In [5]:
len(df.item_id.unique())


1322

In [6]:
len(df.user_id.unique())


44811

In [7]:
len(df)

76000

# Неперсоналізовані рекомендаційні системи

